# Fine Cell Typing

In [ ]:
options(warn = -1, verbose=FALSE)
#!/usr/bin/env Rscript 
library(dplyr)
library(Seurat)
library(httr)
library(readr)
library(pheatmap)
library(RColorBrewer)
library(ggplot2)
library(cowplot)
library(patchwork)
library(unixtools)
library(ggrepel)
library(repr)
library(ggmin)
library(harmony)
library(SeuratWrappers)
library(Nebulosa)
library(ggthemes)
library(purrr)
library(presto)
library(pryr)
set_config(config(ssl_verifypeer = 0L))
ulimit::memory_limit(50000)
set.tempdir("/datastore/lucy/tmp/")
setwd("/datastore/lucy/CosMx")
source("./R/utils.R")
start_upR(clusterfiles = TRUE)

# Load coarse cell typed CosMx data

In [ ]:
objH <- readRDS("./cache/CosMxcoarseGrainharmonyObj_markers.RDS")

In [ ]:
objH$metadata$new.coarse.types <- as.ordered(factor(objH$metadata$new.coarse.types, c("Adipocytes","PRG4+ Lining Fibroblast","Fibroblast","Mural","Endothelial",
                                                                                    "Plasma","B cells","CD4+ Tcell","CD8+ Tcell","NK cell","Mast","Macrophage",
                                                                                    "cDC")))
                                                                    
cols <- c("#ffeca9","gold","sienna1","sienna3","red3","pink","deeppink","green3","green4","darkgreen","lightblue1","royalblue1","turquoise1")
names(cols) <- levels(objH$metadata$new.coarse.types)

In [ ]:
fig.size(6, 10)
plot <- plot_dim_red(
    dim_red_embeddings = objH$Humap$embedding,
    clusters = objH$Humap$clusters,
    metadata = objH$metadata,
    cell_id_colname = "cellID",
    color_by = "new.coarse.types",
    plot_title = "UMAP after Harmony",
    dim_red_type = "UMAP",
    size_points = 0.3,
    plot_labels = FALSE
)
plot + 
  theme(legend.position="none") + theme_void() + scale_fill_manual(values=cols)+ scale_color_manual(values=cols)

# Load cell geoms

In [ ]:
cellgeoms <- readRDS("./cache/cellgeoms_finetyped.RDS")

In [ ]:
#Load niche annotations for each cell
niche_ann <- readRDS("./cache/tissueSegmentation/niche_ann.rds")

In [ ]:
head(niche_ann)

In [ ]:
niche_ann <- niche_ann %>% dplyr::select(cellID, highres.regions)
#Add niche info to cell geoms
cellgeoms <- cellgeoms %>% left_join(niche_ann, by="cellID")

In [ ]:
head(cellgeoms)

In [ ]:
unique(cellgeoms$SampleFOV)

In [ ]:
cellgeoms$centroid <- st_centroid(cellgeoms$shape)

In [ ]:
cols <- c("#ffeca9","gold","sienna1","sienna3","red3","pink","deeppink","green3","green4","darkgreen","lightblue1","royalblue1","turquoise1")
names(cols) <- levels(cellgeoms$new.coarse.types)

In [ ]:
unique(cellgeoms$SampleFOV)

# New coarse cell types in space

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
            geom_sf(aes(geometry = geometry, fill = new.coarse.types, color=new.coarse.types), na.rm=TRUE) + #geom_sf(data = niche_shape) +
            scale_fill_manual(values = cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = cols,na.value="black")+ dark_theme_classic() +
            ggtitle(.x) + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
    )

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
        subset(SampleFOV %in% "Run5666_G7G72_G22G5_F023") %>% 
        ggplot() + 
            geom_sf(aes(geometry = geometry, fill = new.coarse.types, color=new.coarse.types), na.rm=TRUE) + #geom_sf(data = niche_shape) +
            scale_fill_manual(values = cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = cols,na.value="black")+ dark_theme_classic() +
            ggtitle(cellgeoms$SampleFOV) + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) + coord_sf(ylim = c(-6100,-6400), xlim=c(44850,45150), clip = "on")
    

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
            geom_sf(aes(geometry = geometry, fill = new.coarse.types, color=new.coarse.types), na.rm=TRUE) + #geom_sf(data = niche_shape) +
            scale_fill_manual(values = cols, na.value = "lightgrey", na.translate = TRUE) + scale_color_manual(values = cols,na.value="white")+ 
            ggtitle(.x) + guides(fill=FALSE, color=FALSE) + theme_powerpoint() + theme(axis.text=element_blank(), axis.ticks=element_blank())
    )

In [ ]:
cellgeoms$highres.regions <- as.ordered(factor(cellgeoms$highres.regions, levels=c('PRG4+ FLS Lining','Myeloid-rich Lining',
                                                                                 'Sublining',
                                                                                  'Adipocyte-rich','Perivascular',
                                                                                'TGFB+ Myeloid-rich', 'Plasma-rich',
                                                                                  'Lymphoid')))

In [ ]:
unique(cellgeoms$highres.regions)

In [ ]:
highres.cols <- c("gold","#F28E2B","#8CD17D","#FFBC79","#FF534E","pink","hotpink",
                  "#4DCBE8")
names(highres.cols)  <- levels(cellgeoms$highres.regions)

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = highres.regions, color = highres.regions), alpha = 1, linewidth=1, na.rm=TRUE)  +
     scale_fill_manual(values = highres.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = highres.cols)+ ggtitle("")+ theme_powerpoint() + dark_theme_classic() + theme(axis.text=element_blank(), axis.ticks=element_blank()) + guides(fill=FALSE, color=FALSE)
)

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = highres.regions, color = highres.regions), alpha = 1, linewidth=1, na.rm=TRUE)  +
     scale_fill_manual(values = highres.cols, na.value = "white", na.translate = TRUE) + scale_color_manual(values = highres.cols)+ ggtitle("")+ theme_powerpoint() + theme(axis.text=element_blank(), axis.ticks=element_blank()) + guides(fill=FALSE, color=FALSE) 
)

# New coarse cell types illustrating only cDC and CD4 t cell in space

In [ ]:
cellgeoms$coarse.interest <- ifelse(cellgeoms$new.coarse.types%in%c("CD4+ Tcell","cDC"), as.character(cellgeoms$new.coarse.types), NA)

In [ ]:
#Assign levels so clusters appear in order
cellgeoms$fine.types <- as.ordered(factor(cellgeoms$fine.types, 
                        levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',
                                 'Myeloid_DC2 CD1c+_doublet',
                                 
                                 'Myeloid_CLEC10A+ATF3+ STM',
                                 'Myeloid_FOLR2highEGR1+ STM',
                                 'Myeloid_FOLR2highLYVE1+ STM',
                                 'Myeloid_TGFB2+BIRC5+ STM',
                                 'Myeloid_BIRC5+ cycling STM',
                                 'Myeloid_TREM2+ STM',
                                 'Myeloid_TREM2+LPL+ STM',
                                 'Myeloid_SPP1+TREM2low STM',
                                 'Myeloid_SPP1+ STM',
                                 'Myeloid_TNF+ICAM1+ STM',
                                 'Myeloid_ISG15+CXCL10+ STM',
                                 'Myeloid_CD14highS100A12+ monoSTM',
                                 'Myeloid_FCGR3A+ monoSTM',
                                 'Myeloid',
                                 'DC',
                                 'Fibroblasts',
                                 
                                 'Stromal_PRG4+ lining FLS',
                                 'Stromal_PRG4+MMP3+ lining FLS',
                                 'Stromal_THY1+CXCL12+ FLS',
                                 'Stromal_THY1+COL1A1+ FLS',
                                 'Stromal_THY1+CD34+MFAP5+ FLS',
                                 'Stromal_THY1+GAS6+CXCL14+ FLS',
                                 'Stromal_CYP1B1+CD52+ FLS',
                                 'Stromal_C1QA+ FLS doublets', 
                                 
                                 'Stromal_NOTCH3+MCAM+ mural cells',
                                 'Stromal_SPARC+ capillary EC',
                                 'Stromal_NOTCH4+ arteriolar EC',
                                 'Stromal_ICAM1+CLU+ venular EC',
                                 'Stromal_CCL21+LYVE1+ lymphatic EC',
                                 'Stromal_NA',
                                 
                                 "Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_T/FLS doublet',
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs',
                                 'Tcell_ANXA1+HSP+ CD4/CD8',
                                 'Tcell_low counts',
                                 'Tcell_CCL5+CD8B+CD8A+ Naive',
                                 'Tcell_GZMK+CCL5+CD8A+ TEM',
                                 'Tcell_GZMK+CCL5+CD8A+/FLS doublet',
                                 'Tcell_KLRB1+GZMK+ NKT','Tcell_MYOM2+ NK/STM doublet',
                                 'Tcell_MYOM2+ NK', 'Tcell_KLRC2+ NK','Tcell_XCL1+ NK',
                                 
                                
                                 "Adipocytes",
                                 "Plasma","B cells","Mast"
                                )))

In [ ]:
#Assign colour for each cluster
fine.type.cols <- c('Myeloid_DC1 CLEC9A+'="#00689A",
                    'Myeloid_DC2 CD1c+'="#43f782",
                    'Myeloid_DC2 CCR7+'="#C776F8",
                    'Myeloid_DC3 CD1clowCD163+'="#F8A640",
                    'Myeloid_iDC3 CD14highCD163+'="#FF6C67",
                    'Myeloid_FOLR2highCLEC10A+ STM'="yellow",
                    
                                 'Myeloid_CLEC10A+ATF3+ STM'='yellow',
                                 'Myeloid_FOLR2highEGR1+ STM'='#27CAF3',
                                 'Myeloid_FOLR2highLYVE1+ STM'="#9aecfa",
                                  'Myeloid_TGFB2+BIRC5+ STM'='orchid1',

                                 'Myeloid_BIRC5+ cycling STM'='magenta1',
                                 'Myeloid_TREM2+ STM'='dodgerblue2',
                                 'Myeloid_TREM2+LPL+ STM'='dodgerblue2',
                                 'Myeloid_SPP1+TREM2low STM'='#FF070A',
                                 'Myeloid_SPP1+ STM'='#FF070A',
                                 'Myeloid_TNF+ICAM1+ STM'='deeppink4',
                                 'Myeloid_ISG15+CXCL10+ STM'='#EB4691',
                                 'Myeloid_CD14highS100A12+ monoSTM'='#EF90D4',
                                 'Myeloid_FCGR3A+ monoSTM'='darkorchid3',
                                 'Myeloid'="royalblue1",
                                 'DC'="turquoise1",
                                 'Fibroblasts'="gold",
                                 'Stromal_PRG4+ lining FLS'="yellow",
                                 'Stromal_PRG4+MMP3+ lining FLS'="orange",
                                 'Stromal_THY1+CXCL12+ FLS'='red',
                                 'Stromal_THY1+COL1A1+ FLS'="pink",
                                 'Stromal_THY1+CD34+MFAP5+ FLS'="green",
                                 'Stromal_THY1+GAS6+CXCL14+ FLS'='dodgerblue1',
                                 'Stromal_CYP1B1+CD52+ FLS'="gold",
                                 'Stromal_C1QA+ FLS doublets'="dodgerblue1", 
                                 
                                 'Stromal_NOTCH3+MCAM+ mural cells'="#B29A42",
                                 'Stromal_SPARC+ capillary EC'="#D0605E",
                                 'Stromal_NOTCH4+ arteriolar EC'="#E49343",
                                 'Stromal_ICAM1+CLU+ venular EC'="#F5C187",
                                 'Stromal_CCL21+LYVE1+ lymphatic EC'="#5878A3",
                                 'Stromal_NA'="#EBCF73",
                "Tcell_CD55+PDE4B+CD4+ Naive"='#AFE1AF',
"Tcell_CCR7+CD7+CD4+ Naive/TCM"="#7CFC00",
'Tcell_BAG3+DNAJB1+CD4+ T'="Purple",
                         'Tcell_CCR6+RORA+TNFSF13B+ TEM'="darkorange",
                                                      'Tcell_CCL5+GZMA+CD4+ TEM'='#FF3131',
                                                                         'Tcell_CCL5+CXCR6+MAF+ TPH'="dodgerblue2",
                                                                         'Tcell_CXCL13hi MAF+ TPH'="darkturquoise",
                                                                         'Tcell_FOXP3+ Tregs'="#FFFF00",
                                                                         'Tcell_ANXA1+HSP+ CD4/CD8'="#EEDE5A",
                                                                         'Tcell_low counts'="#FDBF6F",
                                                                         'Tcell_CCL5+CD8B+CD8A+ Naive'="#FF7F0E",
                                                                         'Tcell_GZMK+CCL5+CD8A+ TEM'="deeppink1",
                                                                         'Tcell_GZMK+CCL5+CD8A+/FLS doublet'="#D62728",
                                                                         'Tcell_KLRB1+GZMK+ NKT'="maroon",
                                                                         'Tcell_MYOM2+ NK/STM doublet'="#FB9A99",
                                                                         'Tcell_MYOM2+ NK'="#CAB2D6",
                                                                         'Tcell_KLRC2+ NK'="#9467BD",
                                                                         'Tcell_XCL1+ NK'="#6A3D9A",
 "Adipocytes"="#ffeca9",
                                 "Plasma"="pink","B cells"="deeppink","Mast"="lightblue1"
                                )

# cDC finetypes

In [ ]:
head(cellgeoms)

In [ ]:
cellgeoms$dc.niche <- ifelse(cellgeoms$fine.types%in%c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                                                                           'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM'), as.character(cellgeoms$fine.types), as.character(cellgeoms$highres.regions))

In [ ]:
cellgeoms$dc.interest <- ifelse(cellgeoms$fine.types%in%c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                                                                           'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM'), as.character(cellgeoms$fine.types), NA)

In [ ]:
unique(cellgeoms$dc.niche)

In [ ]:
cellgeoms$dc.interest <- as.ordered(factor(cellgeoms$dc.interest, 
                        levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM'
                                )))

In [ ]:
cellgeoms$dc.niche <- as.ordered(factor(cellgeoms$dc.niche, 
                        levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',
                                 levels(cellgeoms$highres.regions)
                                )))

In [ ]:
cellgeoms$t.interest <- ifelse(cellgeoms$fine.types%in%c(
                                 "Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'), as.character(cellgeoms$fine.types), NA)

In [ ]:
levels(cellgeoms$highres.regions)

In [ ]:
outline.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("grey50",2),rep("grey",10))
names(outline.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
levels(cellgeoms$dc.niche)

In [ ]:
lining.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("grey50",2),rep("black",10))
names(lining.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
library(ggdark)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = dc.niche, color = dc.niche), na.rm=TRUE,linewidth = 0.1,size=1)  +
     scale_fill_manual(values = lining.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = outline.cols)+ ggtitle("")+ theme_powerpoint() + dark_theme_classic() + guides(fill=FALSE,color=FALSE)
)

In [ ]:
outline.cols <- c("black","black","black","black",
                "black","black",rep("darkgrey",2),rep("darkgrey",10))
names(outline.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
lining.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("darkgrey",2),rep("white",10))
names(lining.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = dc.niche, color = dc.niche), na.rm=TRUE, linewidth=0.05)  +
     scale_fill_manual(values = lining.cols, na.value = "white", na.translate = TRUE) + scale_color_manual(values = outline.cols)+ ggtitle("")+ theme_powerpoint() + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
outline.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("grey",7),"grey50")
names(outline.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
lymphoid.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("black",7),"grey50")
names(lymphoid.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = dc.niche, color = dc.niche), na.rm=TRUE,linewidth = 0.15)  +
     scale_fill_manual(values = lymphoid.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = outline.cols)+ ggtitle("")+ theme_powerpoint() + dark_theme_classic() + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
outline.cols <- c("black","black","black","black",
                "black","black",rep("grey",7),"darkgrey")
names(outline.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
lymphoid.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("white",7),"darkgrey")
names(lymphoid.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = dc.niche, color = dc.niche), na.rm=TRUE, linewidth=0.05)  +
     scale_fill_manual(values = lymphoid.cols, na.value = "white", na.translate = TRUE) + scale_color_manual(values = outline.cols)+ ggtitle("")+ theme_powerpoint() + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
outline.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("grey",2),"grey50",rep("grey",8))
names(outline.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
myeloidsub.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("black",2),"grey50",rep("black",8))
names(myeloidsub.cols) <- levels(cellgeoms$dc.niche)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = dc.niche, color = dc.niche), na.rm=TRUE,linewidth = 0.15, alpha=3)  +
     scale_fill_manual(values = myeloidsub.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = outline.cols)+ ggtitle("")+ theme_powerpoint() + dark_theme_classic() + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
unique(cellgeoms$t.interest)

In [ ]:
cellgeoms$t.interest <- as.ordered(factor(cellgeoms$t.interest, 
                        levels=c(
                                 "Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'
                                )))

In [ ]:
unique(cellgeoms$

In [ ]:
cols

In [ ]:
cellgeoms$coarse.niche <- ifelse(cellgeoms$new.coarse.types%in%c("CD4+ Tcell","cDC"),
                                    as.character(cellgeoms$new.coarse.types), 
                                    as.character(cellgeoms$highres.regions))

In [ ]:
cellgeoms$coarse.niche <- as.ordered(factor(cellgeoms$coarse.niche, levels=c("cDC","CD4+ Tcell",levels(cellgeoms$highres.regions))))
                                

In [ ]:
levels(cellgeoms$coarse.niche)

In [ ]:
outline.cols <- c('turquoise1',"green4",rep("grey50",10),"grey50","grey50")
names(outline.cols) <- levels(cellgeoms$coarse.niche)

In [ ]:
dc.t.cols <- c('turquoise1',"green4",rep("black",10),"grey50","black")
names(dc.t.cols) <- levels(cellgeoms$coarse.niche)

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = coarse.niche, color=coarse.niche), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = dc.t.cols, na.value = "grey50", na.translate = TRUE) + scale_color_manual(values = outline.cols,na.value="grey50")+ ggtitle(cellgeoms$SampleFOV)+ theme_powerpoint() + guides(fill=FALSE, color=FALSE)+ dark_theme_classic() + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
cellgeoms$t.interest <- as.ordered(factor(cellgeoms$t.interest, 
                        levels=c(
                                 "Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'
                                )))

In [ ]:
unique(cellgeoms$

In [ ]:
cols

In [ ]:
cellgeoms$coarse.niche <- ifelse(cellgeoms$new.coarse.types%in%c("CD4+ Tcell","cDC"),
                                    as.character(cellgeoms$new.coarse.types), 
                                    as.character(cellgeoms$highres.regions))

In [ ]:
levels(cellgeoms$coarse.niche)

In [ ]:
outline.cols <- c('turquoise1',"green4",rep("grey50",10),"grey50","grey50")
names(outline.cols) <- levels(cellgeoms$coarse.niche)

In [ ]:
dc.t.cols <- c('turquoise1',"green4",rep("black",10),"grey50","black")
names(dc.t.cols) <- levels(cellgeoms$coarse.niche)

In [ ]:
library(ggdark)
options(repr.plot.width=10, repr.plot.height=10)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = fine.type.cols, na.value = "grey", na.translate = TRUE) + scale_color_manual(values = fine.type.cols,na.value="grey")+ ggtitle(FOV)+ theme_powerpoint()
)

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c(
                                 'Myeloid_iDC3 CD14highCD163+',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH'), as.character(cellgeoms$fine.types), NA)

In [ ]:
cellgeoms$interest <- as.ordered(factor(cellgeoms$interest, levels=c('Myeloid_iDC3 CD14highCD163+',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH')))
                                

In [ ]:
cols <- c("white","#FF3131","dodgerblue2",rep("black",13))
names(cols) <- levels(cellgeoms$interest)
outline.cols <- c("white","#FF3131","dodgerblue2",rep("grey50",14),rep("black",13))
names(outline.cols) <- levels(cellgeoms$interest)

In [ ]:
library(ggdark)
options(repr.plot.width=10, repr.plot.height=10)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = outline.cols,na.value="grey")+ ggtitle(cellgeoms$SampleFOV)+ theme_powerpoint() + ggtitle("")+ theme_powerpoint() + dark_theme_classic() + guides(fill=FALSE, color=FALSE) + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
library(ggdark)
options(repr.plot.width=10, repr.plot.height=10)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = c("white","dodgerblue2","#FF3131"), na.value = "black", na.translate = TRUE) + scale_color_manual(values = c("white","dodgerblue2","#FF3131"),na.value="grey") +  ggtitle(cellgeoms$SampleFOV)+ theme_powerpoint() + ggtitle("")+ theme_powerpoint() + dark_theme_classic() + theme(axis.text=element_blank(), axis.ticks=element_blank()) 
)

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_T/FLS doublet',
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'), as.character(cellgeoms$fine.types), as.character(cellgeoms$highres.regions))

In [ ]:
cellgeoms$interest <- as.ordered(factor(cellgeoms$interest, levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs','Lining','Lining_Myeloid-rich',
        'Sublining_FLS-rich','Sublining_Myeloid-rich',
        'Sublining_Mast','Adipocyte-rich','Perivascular','Vascular',
        'Plasma-rich','Plasma-rich/Macro',
        'Lymphoid','Myeloid-rich/BIRC5+')))
                                

In [ ]:
levels(cellgeoms$interest)

In [ ]:
outline.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("white",8),rep("grey50",10),"grey50","grey50")
names(outline.cols) <- levels(cellgeoms$interest)

In [ ]:
dc.t.cols <- c("#00689A","#43f782","#C776F8","#F8A640",
                "#FF6C67","yellow",rep("white",8),rep("grey50",2),rep("black",10),"black")
names(dc.t.cols) <- levels(cellgeoms$interest)

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
             scale_fill_manual(values = dc.t.cols, na.value = "grey", na.translate = TRUE) + 
               scale_color_manual(values = outline.cols,na.value="grey") + guides(fill=FALSE, color=FALSE) + 
               dark_theme_classic() + theme(axis.text=element_blank(), axis.ticks=element_blank()))

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_T/FLS doublet',
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'), as.character(cellgeoms$fine.types), NA)

In [ ]:
cellgeoms$interest <- as.ordered(factor(cellgeoms$interest, levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs')))
                                

In [ ]:
#Assign colour for each cluster
fine.type.cols <- c('Myeloid_DC1 CLEC9A+'="white",
                    'Myeloid_DC2 CD1c+'="white",
                    'Myeloid_DC2 CCR7+'="white",
                    'Myeloid_DC3 CD1clowCD163+'="white",
                    'Myeloid_iDC3 CD14highCD163+'="white",
                    'Myeloid_FOLR2highCLEC10A+ STM'="white",
                "Tcell_CD55+PDE4B+CD4+ Naive"='#AFE1AF',
"Tcell_CCR7+CD7+CD4+ Naive/TCM"="#7CFC00",
'Tcell_BAG3+DNAJB1+CD4+ T'="Purple",
                         'Tcell_CCR6+RORA+TNFSF13B+ TEM'="darkorange",
                                                      'Tcell_CCL5+GZMA+CD4+ TEM'='#FF3131',
                                                                         'Tcell_CCL5+CXCR6+MAF+ TPH'="dodgerblue2",
                                                                         'Tcell_CXCL13hi MAF+ TPH'="darkturquoise",
                                                                         'Tcell_FOXP3+ Tregs'="#FFFF00")

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = fine.type.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = fine.type.cols,na.value="grey50")+ ggtitle(cellgeoms$SampleFOV)+ theme_powerpoint() + dark_theme_classic()+ guides(fill=FALSE, color=FALSE) + 
               theme(axis.text=element_blank(), axis.ticks=element_blank())
)

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c(
                                 'Myeloid_DC2 CCR7+',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'), as.character(cellgeoms$fine.types), NA)

In [ ]:
cellgeoms$interest <- as.ordered(factor(cellgeoms$interest, levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs')))
                                

In [ ]:
#Assign colour for each cluster
fine.type.cols <- c('Myeloid_DC1 CLEC9A+'="white",
                    'Myeloid_DC2 CD1c+'="white",
                    'Myeloid_DC2 CCR7+'="white",
                    'Myeloid_DC3 CD1clowCD163+'="white",
                    'Myeloid_iDC3 CD14highCD163+'="white",
                    'Myeloid_FOLR2highCLEC10A+ STM'="white",
                "Tcell_CD55+PDE4B+CD4+ Naive"='#AFE1AF',
"Tcell_CCR7+CD7+CD4+ Naive/TCM"="#7CFC00",
'Tcell_BAG3+DNAJB1+CD4+ T'="Purple",
                         'Tcell_CCR6+RORA+TNFSF13B+ TEM'="darkorange",
                                                      'Tcell_CCL5+GZMA+CD4+ TEM'='#FF3131',
                                                                         'Tcell_CCL5+CXCR6+MAF+ TPH'="dodgerblue2",
                                                                         'Tcell_CXCL13hi MAF+ TPH'="darkturquoise",
                                                                         'Tcell_FOXP3+ Tregs'="#FFFF00")

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = fine.type.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = fine.type.cols,na.value="grey50")+ ggtitle(cellgeoms$SampleFOV)+ theme_powerpoint() + dark_theme_classic()+ guides(fill=FALSE, color=FALSE) + 
               theme(axis.text=element_blank(), axis.ticks=element_blank())
)

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_T/FLS doublet',
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'), as.character(cellgeoms$fine.types), NA)

In [ ]:
cellgeoms$interest <- as.ordered(factor(cellgeoms$interest, levels=c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs')))
                                

In [ ]:
#Assign colour for each cluster
fine.type.cols <- c('Myeloid_DC1 CLEC9A+'="#00689A",
                    'Myeloid_DC2 CD1c+'="#43f782",
                    'Myeloid_DC2 CCR7+'="#C776F8",
                    'Myeloid_DC3 CD1clowCD163+'="#F8A640",
                    'Myeloid_iDC3 CD14highCD163+'="#FF6C67",
                    'Myeloid_FOLR2highCLEC10A+ STM'="yellow",
                "Tcell_CD55+PDE4B+CD4+ Naive"='#AFE1AF',
"Tcell_CCR7+CD7+CD4+ Naive/TCM"="#7CFC00",
'Tcell_BAG3+DNAJB1+CD4+ T'="Purple",
                         'Tcell_CCR6+RORA+TNFSF13B+ TEM'="darkorange",
                                                      'Tcell_CCL5+GZMA+CD4+ TEM'='#FF3131',
                                                                         'Tcell_CCL5+CXCR6+MAF+ TPH'="dodgerblue2",
                                                                         'Tcell_CXCL13hi MAF+ TPH'="darkturquoise",
                                                                         'Tcell_FOXP3+ Tregs'="#FFFF00")

In [ ]:
library(ggdark)
options(repr.plot.width=5, repr.plot.height=5)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = fine.type.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = fine.type.cols,na.value="grey50")+ ggtitle(cellgeoms$SampleFOV)+ theme_powerpoint() + dark_theme_classic()+ guides(color=FALSE) + 
               theme(axis.text=element_blank(), axis.ticks=element_blank())
)

In [ ]:
library(ggdark)
options(repr.plot.width=10, repr.plot.height=10)
cellgeoms %>% 
    with(SampleFOV) %>%  
    unique %>% 
    # head(1) %>% 
    purrr::map(~ cellgeoms %>% 
        subset(SampleFOV %in% .x) %>% 
        ggplot() + 
             geom_sf(aes(geometry = shape, fill = interest, color=interest), na.rm=TRUE,linewidth = 0.15) + 
     scale_fill_manual(values = fine.type.cols, na.value = "black", na.translate = TRUE) + scale_color_manual(values = fine.type.cols,na.value="grey50")+ theme_powerpoint() + dark_theme_classic()+ guides(fill=FALSE, color=FALSE) + 
               theme(axis.text=element_blank(), axis.ticks=element_blank())
)

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c(
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Tcell_FOXP3+ Tregs',                         
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM"), as.character(cellgeoms$fine.types), NA)

In [ ]:
cellgeoms$interest <- ifelse(cellgeoms$fine.types%in%c(
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Tcell_FOXP3+ Tregs',                         
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM"), as.character(cellgeoms$fine.types), NA)